Absolute magnitude is defined to be the apparent magnitude an object would have if it were located at a distance of 10 parsecs.
In astronomy, absolute magnitude (M) is a measure of the luminosity of a celestial object on an inverse logarithmic astronomical magnitude scale.

In [1]:
import time
import numpy as np
import h5py
import hdf5plugin
import pandas as pd
import matplotlib as mpl
import matplotlib.colors as cm
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from unyt import yr, Myr, kpc, arcsec, nJy, Mpc, Msun, erg, s, Hz
from astropy.cosmology import Planck18 as cosmo
from scipy import signal
import os 
import csv
import resource
import pickle
import shutil
import json
from synthesizer.grid import Grid
from synthesizer.parametric import SFH, ZDist
from synthesizer.particle.stars import sample_sfhz
from synthesizer.parametric import Stars as ParametricStars
from synthesizer.particle.particles import CoordinateGenerator
from synthesizer.filters import Filter, FilterCollection
from synthesizer.sed import combine_list_of_seds

from synthesizer.load_data.load_camels import load_CAMELS_IllustrisTNG
from synthesizer.kernel_functions import Kernel

from synthesizer.conversions import lnu_to_absolute_mag

# to clear output
from IPython.display import clear_output

current_directory = os.getcwd()
print("Current directory:", current_directory)
# Start the timer
start_time = time.time()


/tmp/ipykernel_6426/2918952320.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


Current directory: /home/jovyan/camels/play/synth-play/LH


In [2]:

# Alternative method for LF:
# try this method again, but using AB mag instead of mass, and suply your own bins (up to -17, say)
def calc_lf(ab_mag, volume, massBinLimits):
# OG:        hist, dummy = np.histogram(np.log10(mstar), bins = massBinLimits)
        hist, dummy = np.histogram(ab_mag, bins = massBinLimits)
        hist = np.float64(hist)
        phi = (hist / volume) / (massBinLimits[1] - massBinLimits[0])
        phi_sigma = (np.sqrt(hist) / volume) /\
                    (massBinLimits[1] - massBinLimits[0]) # Poisson errors
        return phi, phi_sigma, hist
    


phi_arr =[] #phi
phi_sigma_arr =[] # phi_sigma
hist_arr = [] # hist
z_arr = [] #redshift_074, 
abs_mag_arr = [] #absolute mag (th filter)
Vcom_arr = [] # comoving vol
massBinLimits_arr = []


len(phi_arr)

In [3]:
#directories = [f'LH_{i}' for i in range(1000)]
# TEST:
# LH_50 ,LH_104 skipped. bad data, run again later.
directories = [f'LH_{i}' for i in range(200, 300)]
directories

['LH_200',
 'LH_201',
 'LH_202',
 'LH_203',
 'LH_204',
 'LH_205',
 'LH_206',
 'LH_207',
 'LH_208',
 'LH_209',
 'LH_210',
 'LH_211',
 'LH_212',
 'LH_213',
 'LH_214',
 'LH_215',
 'LH_216',
 'LH_217',
 'LH_218',
 'LH_219',
 'LH_220',
 'LH_221',
 'LH_222',
 'LH_223',
 'LH_224',
 'LH_225',
 'LH_226',
 'LH_227',
 'LH_228',
 'LH_229',
 'LH_230',
 'LH_231',
 'LH_232',
 'LH_233',
 'LH_234',
 'LH_235',
 'LH_236',
 'LH_237',
 'LH_238',
 'LH_239',
 'LH_240',
 'LH_241',
 'LH_242',
 'LH_243',
 'LH_244',
 'LH_245',
 'LH_246',
 'LH_247',
 'LH_248',
 'LH_249',
 'LH_250',
 'LH_251',
 'LH_252',
 'LH_253',
 'LH_254',
 'LH_255',
 'LH_256',
 'LH_257',
 'LH_258',
 'LH_259',
 'LH_260',
 'LH_261',
 'LH_262',
 'LH_263',
 'LH_264',
 'LH_265',
 'LH_266',
 'LH_267',
 'LH_268',
 'LH_269',
 'LH_270',
 'LH_271',
 'LH_272',
 'LH_273',
 'LH_274',
 'LH_275',
 'LH_276',
 'LH_277',
 'LH_278',
 'LH_279',
 'LH_280',
 'LH_281',
 'LH_282',
 'LH_283',
 'LH_284',
 'LH_285',
 'LH_286',
 'LH_287',
 'LH_288',
 'LH_289',
 'LH_290',

In [4]:
len(directories)

100

In [5]:
# Define the path to the log file
log_file_path = "/home/jovyan/camels/play/synth-play/LH/log.txt"

# Initialize the list of skipped directories
skipped_directories = []

# Function to log messages
def log_message(message):
    with open(log_file_path, 'a') as log_file:
        log_file.write(message + '\n')


In [6]:
grid_name = "bc03-2016-Miles_chabrier-0.1,100.hdf5"
grid_dir = "/home/jovyan/"

# Create a new grid
#grid = Grid(grid_name, grid_dir=grid_dir, read_lines=False)

# instead of using a filter, which requires us to load in large SEDs first, pass the grid wavelength
#filt1 = Filter("top_hat/filter.1", lam_min=1400, lam_max=1600, new_lam=grid.lam)
#filt_lst = [filt1]

# Define a new set of wavelengths
lims_lams=(1400, 1600)

grid = Grid(grid_name, grid_dir=grid_dir, lam_lims=lims_lams, read_lines=False)
print(grid.shape)


(221, 7, 401)


In [7]:

# Define the directory where the text files will be saved
output_dir = "/home/jovyan/camels/play/synth-play/LH/output/"


In [8]:
# Loop over multiple LH directories
for LH_X in directories:
    try:
        log_message(f"Processing directory: {LH_X}")

        # Get gals
        dir_ = '/home/jovyan/Data/Sims/IllustrisTNG/LH/' + LH_X

        try:
            gals_074 = load_CAMELS_IllustrisTNG(
                dir_,
                snap_name='snapshot_074.hdf5', 
                fof_name='groups_074.hdf5',
                verbose=False  # Suppress output
            )
        except Exception as e:
            log_message(f"Error loading CAMELS data for {LH_X}: {e}")
            skipped_directories.append(LH_X)
            continue

        # Clear the output after each iteration
        clear_output(wait=True)

        try:
            cat_074 = dir_+'/groups_074.hdf5'
            # Open file
            f_h5py = h5py.File(cat_074, 'r')

            # Read different attributes of the header
            boxSize_074 = f_h5py['Header'].attrs[u'BoxSize']/1e3 # Mpc/h
            redshift_074 = f_h5py['Header'].attrs[u'Redshift']
        except Exception as e:
            log_message(f"Error reading HDF5 file for {LH_X}: {e}")
            skipped_directories.append(LH_X)
            continue

        # Filter galaxies to only include those with 100 or more star particles
        try:
            gals_074 = [gal for gal in gals_074 if len(gal.stars.ages) >= 100]
            spec_list = []
            # Lets work with z=0 so gals_025
            for i in gals_074:
                gal = i
                # Get spectra incident An Sed object containing the stellar spectra
                spec = gal.stars.get_spectra_incident(grid)
                spec.get_fnu0()
                spec_list.append(spec)

            # Combine
            seds = combine_list_of_seds(spec_list)
            filt1 = Filter("top_hat/filter.1", lam_min=1400, lam_max=1600, new_lam=grid.lam)
            filt_lst = [filt1]
            seds.lnu # Rest frame lumd
            seds.get_photo_luminosities(filt_lst)
            seds.photo_luminosities.photo_luminosities
            abs_mag = lnu_to_absolute_mag(seds.photo_luminosities.photo_luminosities)
            abs_mag_th = abs_mag[0]
            # Next steps, get luminosity function for these magnitudes
            # Co-moving volume: BoxSize_025 and redshift:
            little_h =  0.6711
            Vphys = (boxSize_074/little_h )**3
            Vcom = Vphys * ((1+redshift_074)**3)

            massBinLimits = np.arange(-22, -16, 0.5)
            phi, phi_sigma, hist = calc_lf(abs_mag_th, Vcom, massBinLimits)
            # NOTE: 074 is the same redshift as CV_0/025
            massBinLimits = massBinLimits[:-1]
        except Exception as e:
            log_message(f"Error processing galaxy data for {LH_X}: {e}")
            skipped_directories.append(LH_X)
            continue

        # Define output file path
        output_file = f"{output_dir}{LH_X}.txt"

        try:
            # Write the data to the text file line by line
            with open(output_file, 'w') as txtfile:
                # Write phi values
                txtfile.write("phi\n")
                for value in phi:
                    txtfile.write(f"{value}\n")

                # Write phi_sigma values
                txtfile.write("phi_sigma\n")
                for value in phi_sigma:
                    txtfile.write(f"{value}\n")

                # Write hist values
                txtfile.write("hist\n")
                for value in hist:
                    txtfile.write(f"{value}\n")

                # Write massBinLimits values
                txtfile.write("massBinLimits\n")
                for value in massBinLimits:
                    txtfile.write(f"{value}\n")
            
            log_message(f"Written out: {output_file}")
        except Exception as e:
            log_message(f"Error writing output file for {LH_X}: {e}")
            skipped_directories.append(LH_X)
            continue

        # Clear the terminal output after each iteration
        os.system('cls' if os.name == 'nt' else 'clear')       

    except Exception as e:
        log_message(f"Error processing {LH_X}: {e}")
        skipped_directories.append(LH_X)

# Write the skipped directories to the log file at the end
if skipped_directories:
    log_message("Skipped directories:")
    for dir in skipped_directories:
        log_message(dir)

KeyboardInterrupt: 

# Try data frame instead
grid_name = "bc03-2016-Miles_chabrier-0.1,100.hdf5"
grid_dir = "/home/jovyan/"
grid = Grid(grid_name, grid_dir=grid_dir, read_lines=False)

# Define the directory where the text files will be saved
output_dir = "/home/jovyan/camels/play/synth-play/LH/output/"

for LH_X in directories:
    try:
        # get grid for all galaxies
        # gives grid of metalicity and age which is used to map on our camels galaxies

        # get gals
        dir_ = '/home/jovyan/Data/Sims/IllustrisTNG/LH/' + LH_X
        gals_074 = load_CAMELS_IllustrisTNG(
            dir_,
            snap_name='snapshot_074.hdf5', 
            fof_name='groups_074.hdf5',
            verbose=False  # Suppress output
        )
        # Clear the output after each iteration
        clear_output(wait=True)

        cat_074 = dir_+'/groups_074.hdf5'
        # open file
        f_h5py = h5py.File(cat_074, 'r')

        # read different attributes of the header
        boxSize_074 = f_h5py['Header'].attrs[u'BoxSize']/1e3 #Mpc/h
        redshift_074 = f_h5py['Header'].attrs[u'Redshift']

        # Filter galaxies to only include those with 100 or more star particles
        gals_074 = [gal for gal in gals_074 if len(gal.stars.ages) >= 100]
        spec_list = []
        # Lets work with z=0 so gals_025
        for i in gals_074:
            gal = i
            # get_spectra_incident An Sed object containing the stellar spectra
            spec = gal.stars.get_spectra_incident(grid)
            spec.get_fnu0()
            spec_list.append(spec)

        # combine
        seds = combine_list_of_seds(spec_list)
        filt1 = Filter("top_hat/filter.1", lam_min=1400, lam_max=1600, new_lam=grid.lam)
        filt_lst = [filt1]
        seds.lnu # rest frame lumd
        seds.get_photo_luminosities(filt_lst)
        seds.photo_luminosities.photo_luminosities
        abs_mag = lnu_to_absolute_mag(seds.photo_luminosities.photo_luminosities)
        abs_mag_th = abs_mag[0]
        # next steps, get luminosity function for these magnitudes
        # co-moving volume: BoxSize_025 and redshift:
        little_h =  0.6711
        Vphys = (boxSize_074/little_h )**3
        Vcom = Vphys * ((1+redshift_074)**3)

        massBinLimits = np.arange(-22, -16, 0.5)
        phi, phi_sigma, hist = calc_lf(abs_mag_th, Vcom, massBinLimits)
        # NOTE: 074 is the same redshift as CV_0/025
        massBinLimits = massBinLimits[:-1]

        # Define output file path
        output_file = f"{output_dir}{LH_X}.txt"

        # Write the data to the text file line by line
        with open(output_file, 'w') as txtfile:
            # Write phi values
            txtfile.write("phi\n")
            for value in phi:
                txtfile.write(f"{value}\n")

            # Write phi_sigma values
            txtfile.write("phi_sigma\n")
            for value in phi_sigma:
                txtfile.write(f"{value}\n")

            # Write hist values
            txtfile.write("hist\n")
            for value in hist:
                txtfile.write(f"{value}\n")

            # Write massBinLimits values
            txtfile.write("massBinLimits\n")
            for value in massBinLimits:
                txtfile.write(f"{value}\n")

        print('Written out: /home/jovyan/camels/play/synth-play/LH/output/',LH_X)

        # Clear the terminal output after each iteration
        os.system('cls' if os.name == 'nt' else 'clear')       
        
    except Exception as e:
        error_message = f"Error processing {LH_X}: {e}"
        print(error_message)
        skipped_directories.append(LH_X)

        # Append the error message to the log file
        with open(log_file_path, 'a') as log_file:
            log_file.write(error_message + '\n')

# Write the skipped directories to the log file at the end
if skipped_directories:
    with open(log_file_path, 'a') as log_file:
        log_file.write("Skipped directories:\n")
        for dir in skipped_directories:
            log_file.write(f"{dir}\n")

In [ ]:
# Stop the timer
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

print("Elapsed time:", elapsed_time, "seconds")

In [ ]:
# Get memory usage in bytes
memory_usage = resource.getrusage(resource.RUSAGE_SELF).ru_maxrss
print("Memory usage end:", memory_usage/1000000, "GB")


# will do latest LH_ set in loop as a test
label_z = 'z = 0.46'


# Plot the luminosity function
plt.errorbar(massBinLimits, phi, yerr=phi_sigma, fmt='o', color='blue',label=label_z)
plt.xlabel('Absolute Magnitude (AB)')
plt.ylabel('Number Density (Mpc^-3 mag^-1)')
plt.yscale('log')

plt.title('Luminosity Function XMM-OM filter')
plt.grid(True)
plt.show()